In [1]:
import os

In [9]:
%pwd

'/home/aman/Desktop/TextCraft'

In [3]:
os.chdir("../")

In [5]:
from pathlib import Path
from dataclasses import dataclass

In [24]:
@dataclass(frozen=True)
class DataIngectionConfig:
    root_dir: Path
    source_URL: str
    local_path: Path
    unzip_dir: Path

In [8]:
from TextCraft.constants import *
from TextCraft.utils.common import read_yaml, create_directories

In [25]:
class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngectionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngectionConfig(
            root_dir=config.root_dir,
            source_URL = config.source_URL,
            local_path = config.local_path,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config

In [26]:
import os
import urllib.request as request
from TextCraft.logging import logger
from TextCraft.utils.common import get_size
import zipfile

In [32]:
class DataIngestion:
    def __init__(self, config: DataIngectionConfig):
        self.config = config
    
    def download(self):
        if not os.path.exists(self.config.local_path):
            filename, headers = request.urlretrieve(
                url= self.config.source_URL,
                filename = self.config.local_path
            )
            logger.info(f"{filename} is Download! Sucessfully!")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_path))}")
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file info into the data directory
        function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [37]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion_config = DataIngestion(config = data_ingestion_config)
    data_ingestion_config.download()
    data_ingestion_config.extract_zip_file()
except Exception as e:
    raise e

[2024-09-08 20:02:45,255: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-08 20:02:45,259: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-08 20:02:45,261: INFO: common: created directory at: artifacts]
[2024-09-08 20:02:45,264: INFO: common: created directory at: artifacts/data_ingestion]
[2024-09-08 20:02:45,266: INFO: 857418747: File already exists of size: ~ 49423 KB]
